#### Code to perform inference on validation data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

## Imports

In [ ]:
import os
import cv2
import json
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from params import *

from data.preparation import prepare_dataframe, handle_duplicates, add_additional_boxes

from data.dataset import CovidInfDataset
from data.transforms import get_tranfos_inference

from utils.logger import Config

from inference.main import inference_val

## Data

## Dataset

In [ ]:
root = DATA_PATH + f"train_{SIZE}/"

In [ ]:
df = prepare_dataframe().copy()

In [ ]:
clusts = np.load("../output/clusts.npy", allow_pickle=True)
found = np.load("../output/found.npy")
transpositions = np.load("../output/transpositions.npy", allow_pickle=True)

df = handle_duplicates(df, clusts, transpositions, plot=False)

In [ ]:
df = add_additional_boxes(df)

In [ ]:
transforms = get_tranfos_inference()
dataset = CovidInfDataset(df, root, transforms=transforms)

In [ ]:
img = dataset[0]
plt.imshow((img.numpy().transpose(1, 2, 0) + 1) / 2)
plt.show()

# Main

In [ ]:
# EXP_FOLDER = LOG_PATH + "2021-07-30/4/"
EXP_FOLDER = LOG_PATH + "2021-07-31/0/"

FLIP_TTA = True

config = Config(json.load(open(EXP_FOLDER + "config.json", 'r')))
weights = sorted(glob.glob(EXP_FOLDER + "*.pt"))

In [ ]:
pred_oof_study, pred_oof_img = inference_val(
    config,
    EXP_FOLDER,
    df,
    root_dir=root,
    flip_tta=FLIP_TTA,
)